In [67]:
import pandas as pd
df=pd.read_csv("all_kindle_review .csv")

In [68]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [69]:
df=df[['reviewText','rating']]

In [70]:
df

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,4
11996,I have read all seven books in this series. Ap...,5
11997,This book really just wasn't my cuppa. The si...,3
11998,"tried to use it to charge my kindle, it didn't...",1


In [71]:
df.shape

(12000, 2)

In [72]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [73]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [74]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [75]:
# preprocessing and cleaning


In [76]:
## positive review is 1 and negative review is 0
df['rating']=df['rating'].apply(lambda z:0 if z<3 else 1)

In [77]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [78]:
## lower all the cases
df['reviewText']=df['reviewText'].str.lower()

In [79]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [80]:
import re
from nltk.corpus import stopwords

# 1. Pre-load stopwords (much faster than loading inside the loop)
stop_words = set(stopwords.words('english'))

# 2. Define the cleaning steps
def clean_text(text):
    # 1. Remove HTML tags (Do this first!)
    # Matches anything between < and >
    text = re.sub(r'<[^>]+>', '', text)
    
    # 2. Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # 3. Remove special characters (Keep spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]+', '', text)
    
    # 4. Convert to lowercase and remove stopwords
    text = " ".join([word for word in text.lower().split() if word not in stop_words])

    # 5. Remove any additional spaces (multiple spaces become one)
    # \s+ matches one or more whitespace characters
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# 3. Apply to dataframe
df['reviewText'] = df['reviewText'].apply(clean_text)

In [81]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [82]:
## lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [83]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [84]:
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))

In [85]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [86]:
## train tst split
from  sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],test_size=0.2)

In [87]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train)
X_test_bow=bow.transform(X_test)

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train)
X_test_tfidf=tfidf.transform(X_test)

In [89]:
X_test_bow

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 105424 stored elements and shape (2400, 37311)>

In [90]:
from sklearn.naive_bayes import GaussianNB

# 1. Initialize the model
nb_model_bow = GaussianNB()

# 2. Fit the model using TRAIN data for both X and y
# We use .toarray() to convert the sparse matrix to a dense one for GaussianNB
nb_model_bow.fit(X_train_bow.toarray(), y_train)

# 3. Predict using TEST data
predictions = nb_model_bow.predict(X_test_bow.toarray())

In [91]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [94]:
confusion_matrix(y_test,predictions)

array([[533, 284],
       [682, 901]])

In [93]:
print("BOW :",accuracy_score(y_test,predictions))

BOW : 0.5975
